# Develop and publish tools - API

A common pattern for tools is to connect RESTful APIs with LLM Agents. This pattern is used in the `xentropy-geocoding` tool where the Google Map API is used to perform geocoding.

In this notebook we will first look at how to develop a tool that connects to an API, using geocoding as an example. Then we will look at how to publish a tool to the XEntropy platform with a self hosted API.

In [1]:
from siumai.client import Client

In [ ]:
client = Client(api_key='YOUR_XENTROPY_API_KEY')

In [ ]:
# Creating a tool by integrating with an API

from pydantic import BaseModel
import os
import requests


class Address(BaseModel):
    address: str


class Coordinate(BaseModel):
    latitude: float
    longitude: float


def geocoding(address: Address):
    
    geocoding = requests.get(
        'https://maps.googleapis.com/maps/api/geocode/json',
        params={
            'address': address.address,
            # YOUR Google Cloud API Key
            'key': os.environ.get('GOOGLE_CLOUD_API_KEY')
        }
    ).json()
    location = geocoding.get('results')[0].get('geometry').get('location')
    result = {'latitude': location.get(
        'lat'), 'longitude': location.get('lng')}

    return result


In [ ]:

from siumai.tool import Tool
import json

# Create the tool by overriding the Tool class with the following methods
class GeocodingTool(Tool):
    # call the api here
    def run(**kwargs):
        return json.dumps(
            geocoding(
                Address.model_validate(kwargs)
            )
        )
    
    # optionally implement the asynchronous version of the tool
    async def arun(**kwargs):
        return json.dumps(
            geocoding(
                Address.model_validate(kwargs)
            )
        )
    
# define the tool
geocoding_tool = GeocodingTool(
    name='geocoding',
    description='A geocoding API that converts an address to a coordinate',
    input_json_schema=Address.model_json_schema(),
    output_schema=Coordinate.model_json_schema()
)

In [ ]:
# Test if the code snippet works
geocoding_tool.run(address='1600 Amphitheatre Parkway, Mountain View, CA')

LLM agents can struggle to carry out deterministic logics and complex arithmetics, as the token generation process is stochastic in nature. However, we can develop tools that can carry out such tasks and equip them to LLM agents. In the following section we will look at how to develop such tools and publish to XEntropy platform.

In [ ]:
from hashlib import sha256
from dotenv import load_dotenv
import os
# Developing the geodesic calculation tool used in the quickstart
# XEntropy will send a Webhook-Secret header to your endpoint for authentication.
load_dotenv()

webhook_secret = sha256(b'YOUR OWN SEED').hexdigest()

# Replace with your own google cloud api key
env = {
    "WEBHOOK_SECRET": webhook_secret,
}

with open('.env', 'w') as f:
    f.write('\n'.join([f'{key}={value}' for key, value in env.items()]))

In [ ]:
from pydantic import BaseModel
from geopy import distance

# Code snippet for wrapping some custom logic into a tool to be published on XEntropy
class Coordinate(BaseModel):
    latitude: float
    longitude: float


class CoordinatePair(BaseModel):
    coordinate_0: Coordinate
    coordinate_1: Coordinate

def geodesic(coordinate_pair: CoordinatePair):

    geodesic_distance = distance.distance(
        tuple(coordinate_pair.coordinate_0.dict().values()),
        tuple(coordinate_pair.coordinate_1.dict().values()),
    ).km

    result = {'geodesic_distance': geodesic_distance, 'unit': 'km'}

    return result

In [ ]:
# test the function
geodesic(
    CoordinatePair(
        coordinate_0=Coordinate(latitude=0, longitude=0), 
        coordinate_1=Coordinate(latitude=1, longitude=1)
    )
)

Execute the following to start a FastAPI server on your virtual machine.
```bash
uvicorn server:app --host 0.0.0.0 --port 80 --reload
```

In [ ]:
import requests
# Test if the server is working
url = f'http://IP_OF_YOUR_VIRTUAL_MACHINE/geodesic'
response = requests.post(
    url,
    json={
        'coordinate_0': {'latitude': 40.6446245, 'longitude': -73.7797035},
        'coordinate_1': {'latitude': 41.6446245, 'longitude': -72.7797035}
    },
    headers={
        'Webhook-Secret': webhook_secret
    }
)
response.json()

In [ ]:

from siumai.tool import Tool
from pydantic import BaseModel

geodesic = Tool(
    api_key='YOUR OWN XENTROPY API KEY',
    name='geodesic',
    description='Calculate the earth surface distance between two latitude and longitude coordinate',
    endpoint=url,
    input_model=CoordinatePair,
    price=0,  # xentropy_credit per request. 0 means free to use. 1 USD = 100,000 xentropy_credit
)

tool_upload = geodesic.publish(webhook_secret=webhook_secret, public=True)

In [ ]:
# confirm that the tool works
geodesic = Tool.load(
    tool_upload['name'],
    api_key=os.environ.get('XENTROPY_API_KEY')
)
coordinatePair = CoordinatePair(coordinate_0=Coordinate(latitude=22.3193039, longitude=114.1693611), coordinate_1=Coordinate(
    latitude=35.6764225, longitude=139.650027))

print(geodesic.run(**coordinatePair.model_dump()))
print(await geodesic.arun(**coordinatePair.model_dump()))